In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import image_dataset_from_directory

tf.config.list_physical_devices('GPU')

In [ ]:
train_dir = '../Generator/dataset1/train'
test_dir = '../Generator/dataset1/test'

In [ ]:
IMG_SIZE = 500
BATCH_SIZE = 32
RANDOM_SEED = random.randint(0, 1000)
print(RANDOM_SEED)

In [ ]:
train_dataset = image_dataset_from_directory(
    train_dir,
    #labels='inferred',
    #validation_split=0.2,
    #subset="training",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    seed=RANDOM_SEED
    )

test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    seed=RANDOM_SEED
    )


In [ ]:
for image_batch, labels_batch in train_dataset:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomBrightness(0.1),
    layers.RandomContrast(0.1),
    layers.RandomCrop(IMG_SIZE, IMG_SIZE)
])

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Conv2D(filters=64,kernel_size=(11,11),strides=(4,4),padding='valid',activation='relu')(x)
# Layer 2: Max pooling layer with pool size of 3x3
x = layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)

# Layer 3-5: 3 more convolutional layers with similar structure as Layer 1
x = layers.Conv2D(filters=192,kernel_size=(5,5),padding='same',activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)
x = layers.Conv2D(filters=384,kernel_size=(3,3),padding='same',activation='relu')(x)
x = layers.Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(3,3),strides=(2,2))(x)

# Layer 6: Fully connected layer with 4096 neurons
x = layers.Flatten()(x)
x = layers.Dense(4096, activation="relu")(x)

# Layer 7: Fully connected layer with 4096 neurons
x = layers.Dense(4096, activation="relu")(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), metrics=['acc'])

In [ ]:
history = model.fit(train_dataset, epochs=20, batch_size=32)

In [ ]:
img = np.array([])
for data_batch, label in test_dataset.take(1):
        image = data_batch[random.randint(0,10)].numpy().astype("uint8")
        img = np.append(img, image)
        plt.imshow(image)
        plt.show()
        break
img = img.reshape(IMG_SIZE, IMG_SIZE, 3)
img.astype("float32") / 255
img = np.expand_dims(img, axis=0)

print(model.predict(img).round())

In [ ]:
val_loss, val_acc = model.evaluate(test_dataset, batch_size=16)
print(val_loss, val_acc)

In [ ]:
acc = history.history['acc']
loss = history.history['loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')

ax = plt.gca()
ax.set_ylim([0, 1])
plt.title('Training accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
img = tf.keras.preprocessing.image.load_img('cats_and_dogs_small/validation/dogs/dog.1010.jpg', target_size=(150, 150), interpolation='bilinear')

In [ ]:
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
print(img_array.shape)
result = model.predict(img_array)
print("Result: ", result.round())